In [1]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
    combine_train_datasets
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
import core
import keras
import tensorflow as tf


PLOTS_DIR = f"plots/mixed_model/"
MODEL_DIR = f"models/mixed_model"
#MODEL_DIR = f"../models/FeatureConcatTransformer_HLF_d256_l10_h8"
CONFIG_PATH = "../config/workspace_config_HLF.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR, exist_ok=True)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR, exist_ok=True)

load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessorNominal = DataPreprocessor(load_config)
DataProcessorToponium = DataPreprocessor(load_config)

with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)

plt.rcParams.update({"font.size": 14})

data_config = DataProcessorNominal.load_from_npz(data_configs["data_path"]["nominal"], max_events=2000000)
DataProcessorToponium.load_from_npz(data_configs["data_path"]["toponium"])

X_nominal, y_nominal = DataProcessorNominal.get_data()
X_toponium, y_toponium = DataProcessorToponium.get_data()
X_train, y_train = combine_train_datasets(
    [X_nominal, X_toponium],
    [y_nominal, y_toponium],
    weights=[0.5, 0.5],
)

2025-11-26 12:38:14.558727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764157094.581014 3860220 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764157094.588402 3860220 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764157094.606684 3860220 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764157094.606704 3860220 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764157094.606706 3860220 computation_placer.cc:177] computation placer alr

In [ ]:
reload(Models)
reload(core)
TransformerMatcher = Models.FeatureConcatTransformer(data_config, name=r"Transformer + $\nu^2$-Flows")

#TransformerMatcher.load_model(f"{MODEL_DIR}/model.keras")
TransformerMatcher.build_model(
    hidden_dim=64,
    num_layers=6,
    num_heads=8,
    dropout_rate=0.1,
)

TransformerMatcher.adapt_normalization_layers(X_train)

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(lambda_excl=0.0),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
    callbacks=[
        keras.callbacks.ModelCheckpoint(
            filepath=f"{MODEL_DIR}/model.keras",
            save_best_only=True,
            monitor="val_assignment_AssignmentAccuracy",
            mode="max",
            verbose=1,
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_assignment_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
    ],
)
TransformerMatcher.model.summary()

FeatureConcatTransformer is designed for classification tasks; regression targets will be ignored.


I0000 00:00:1764157198.755613 3860220 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.


Model: "FeatureConcatTransformerModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ met_inputs          │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_transform │ (None, 1, 2)      │          0 │ met_inputs[0][0]  │
│ (InputMetPhiLayer)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_transform │ (None, 2, 4)      │          0 │ lep_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_inputs          │ (None, 6, 9)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_normaliz… │ (None, 1, 2)      │          5 │ met_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_normaliz… │ (None, 2, 4)      │          9 │ lep_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_transform │ (None, 6, 9)      │          0 │ jet_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 2)         │          0 │ met_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 8)         │          0 │ lep_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_normaliz… │ (None, 6, 9)      │         19 │ jet_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 6, 2)      │          0 │ flatten[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_1     │ (None, 6, 8)      │          0 │ flatten_1[0][0]   │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 19)     │          0 │ jet_input_normal… │
│ (Concatenate)       │                   │            │ repeat_vector[0]… │
│                     │                   │            │ repeat_vector_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding (MLP) │ (None, 6, 256)    │     33,470 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6, 1)      │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 256)    │    527,104 │ jet_embedding[0]… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 3,209,509 (12.24 MB)

 Trainable params: 3,209,476 (12.24 MB)

 Non-trainable params: 33 (144.00 B)

In [3]:
TransformerMatcher.train_model(
    epochs=100,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
)

Epoch 1/100


I0000 00:00:1764157222.717716 3862502 service.cc:152] XLA service 0x7fb5dc00f3f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764157222.717746 3862502 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-11-26 12:40:23.385781: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-26 12:40:24.044528: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
I0000 00:00:1764157226.033535 3862502 cuda_dnn.cc:529] Loaded cuDNN version 91500


   2/2753 ━━━━━━━━━━━━━━━━━━━━ 3:59 87ms/step - accuracy: 0.1406 - loss: 0.0014   

I0000 00:00:1764157240.697041 3862502 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2752/2753 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.3676 - loss: 8.4990e-04

2025-11-26 12:44:28.947302: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


2753/2753 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.3677 - loss: 8.4987e-04

2025-11-26 12:44:45.794140: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-11-26 12:44:56.512232: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


2753/2753 ━━━━━━━━━━━━━━━━━━━━ 290s 93ms/step - accuracy: 0.3677 - loss: 8.4983e-04 - val_accuracy: 0.6311 - val_loss: 6.0363e-04
Epoch 2/100
2753/2753 ━━━━━━━━━━━━━━━━━━━━ 236s 86ms/step - accuracy: 0.6065 - loss: 6.4129e-04 - val_accuracy: 0.6858 - val_loss: 5.6086e-04
Epoch 3/100
2753/2753 ━━━━━━━━━━━━━━━━━━━━ 236s 86ms/step - accuracy: 0.6552 - loss: 6.0064e-04 - val_accuracy: 0.7116 - val_loss: 5.4718e-04
Epoch 4/100
2753/2753 ━━━━━━━━━━━━━━━━━━━━ 236s 86ms/step - accuracy: 0.6757 - loss: 5.8087e-04 - val_accuracy: 0.7232 - val_loss: 5.3158e-04
Epoch 5/100
2753/2753 ━━━━━━━━━━━━━━━━━━━━ 236s 86ms/step - accuracy: 0.6866 - loss: 5.6977e-04 - val_accuracy: 0.7253 - val_loss: 5.2603e-04
Epoch 6/100
2753/2753 ━━━━━━━━━━━━━━━━━━━━ 236s 86ms/step - accuracy: 0.6931 - loss: 5.6310e-04 - val_accuracy: 0.7287 - val_loss: 5.2200e-04
Epoch 7/100
2753/2753 ━━━━━━━━━━━━━━━━━━━━ 237s 86ms/step - accuracy: 0.6986 - loss: 5.5589e-04 - val_accuracy: 0.7258 - val_loss: 5.2047e-04
Epoch 8/100
2753/2

KeyboardInterrupt: 

In [5]:
TransformerMatcher.model.save(f"{MODEL_DIR}/model.keras")
#TransformerMatcher.save_model(f"{MODEL_DIR}/model.keras")